In [ ]:
import fiftyone as fo
import json
from pathlib import Path
import fiftyone.utils.coco as fouc
from PIL import Image
from fiftyone import ViewField as F
import numpy as np

In [ ]:

name = 'dental_rtg_dataset'
dataset = fo.load_dataset(name)
dataset.delete()

In [ ]:
name = 'dental_rtg_dataset'
data_path = "/datagrid/personal/kuntluka/dental_rtg3/images"
labels_path = "/datagrid/personal/kuntluka/dental_rtg3/annotations.json"
dataset = fo.Dataset.from_dir(
    dataset_type = fo.types.COCODetectionDataset,
    data_path = data_path,
    labels_path = labels_path,
    name=name
)


In [ ]:
session = fo.launch_app(dataset,auto=False)

In [ ]:
session.dataset = dataset


In [ ]:
session.view = dataset.view()

In [ ]:
with open('val_seg_names.json','r') as f:
    filenames = json.load(f)

In [ ]:
data = np.load('res_mask.npy')
# np.min(data[0,:,:])
data = data >= 0.5

In [ ]:
data.shape

In [ ]:
for i, m in enumerate(data):
    f = str(i+1) + ".png"
    sample = dataset[str(Path(data_path)/f)]
    sample['segmentation'] = fo.Segmentation(mask=m)
    sample.tags.append('seg')
    sample.save()
    

In [ ]:
m = np.zeros_like(data[0])

In [ ]:
m[data[0]] = 2

In [ ]:
s = fo.Segmentation(mask=m)

In [ ]:
view = dataset.select_by("filepath", str(Path(data_path)/'169.png'))

In [ ]:
view = dataset[str(Path(data_path)/'150.png')]
view['segmentation'] = s
view.save()



In [ ]:
for i in range(25):
    test_v = dataset.match_tags('test')
    test_v = test_v.filter_labels(pred_name, F('confidence')>0.294)
    

In [ ]:
pred_name = 'ens_all_mix'
test_v = dataset.match_tags('train')
test_v = test_v.filter_labels(pred_name, F('confidence')>0.294)
results = test_v.evaluate_detections(
    pred_name,
    gt_field="ground_truth",
    iou=0.5,
    
    compute_mAP = True,
    eval_key='None'
    
)
results.print_report()

In [ ]:
results.metrics()['recall']

In [ ]:
match = (F('eval_ens_all_fn') + F('eval_ens_all_fp'))
mistakes = dataset.match(match > 0)

In [ ]:
results.confusion_matrix()
# results.report()

In [ ]:
session.view = mistakes.sort_by(match, reverse=True)

In [ ]:
anno_key = "p7"

label_schema = {
    "ground_truth" : {
        "type" : "detections",
        "classes" : ["decay"]
    },
    "retinanet_swint" : {
        "type" : "detections",
        "classes" : ["decay"]
    }
}

# Upload the samples and launch CVAT
annot_res = mistakes[800:].annotate(
    anno_key=anno_key,
    label_schema=label_schema,
    attributes=False,
    image_quality=100,
    segment_size=100
)

In [ ]:
def add_prediction(pred_dict, dataset, img_root, name, confidence_threshold=0):
    # for sample in dataset:
    det_id = 0
    for pred_key, pred_value in pred_dict.items():
        dets = []
        # id = sample.id
        path = img_root / pred_key
        sample = dataset[str(path)]
        s_id = sample.id
        scores = pred_value['scores']
        labels = pred_value['labels']
        bboxes = pred_value['bboxes']
        img = Image.open(path)
        for box, score, label in zip(bboxes, scores, labels):
            #TODO to limit confidence
            if score < confidence_threshold:
                continue
            coco_obj = fouc.COCOObject(det_id, s_id, int(label), [box[0],box[1], box[2]-box[0], box[3]-box[1]], score=score)
            det = coco_obj.to_detection(img.size, ['background','decay'])
            dets.append(det)
            det_id += 1
        detection = fo.Detections(detections=dets)
        sample[name] = detection
        sample.tags.append(pred_value['stage'])
#         sample.tags = [pred_value['stage']]
        sample.save()

In [ ]:
# names = ['faster_rcnn_res50','yolov5_medium','faster_rcnn_res101', 'yolov5_large', 'retinanet_swint', 'efficientdet_d4', 'retinanet_resnet50']
names = ['predictions_3500']
data_path = "/datagrid/personal/kuntluka/dental_rtg3/images/"
for name in names:
    with open('predictions_' + name + '.json', 'r') as json_file:
        preds = json.load(json_file)
    add_prediction(preds, dataset, Path(data_path), name)

In [ ]:
with open('predictions_4000/predictions_retinanet_swint.json', 'r') as json_file:
# with open('ens_yolomix_preds.json', 'r') as json_file:
    preds = json.load(json_file)
add_prediction(preds, dataset, Path(data_path), 'swint', confidence_threshold=0.01)